In [ ]:
import xarray as xr
import hvplot.xarray
import dask
from dask.distributed import Client, progress
#from dask_kubernetes import KubeCluster
import os
import fsspec

In [ ]:
from pangeoLib.aws_authenticate import aws_authenticate

aws_authenticate()

In [ ]:
fs = fsspec.filesystem('s3', anon=False, requester_pays=True)

In [ ]:
file_objects = fs.ls('dev-et-data/tiles/')
for file_obj in file_objects:
    if '_o' in file_obj:
        print(file_obj)

In [ ]:
def get_etasw_tifs(bucket_prefix_path):
    files = fs.ls(bucket_prefix_path)
    tifs = [f for f in files if f.endswith('tif')]
    etasw_list = [f for f in tifs if 'etasw_' in f]
    return(etasw_list)

In [ ]:
chip='dev-et-data/tiles/tile40N-110E_chip40N-110E_o'
ets = get_etasw_tifs(chip)

In [ ]:
ets=[]

for i in range(1,365+1):
    day = '{:03}'.format(i)
    #print(day)
    tif = chip + '/' + 'etasw_2014' + day + '.tif'
    #print(tif)
    ets.append(tif)
    

In [ ]:
os.environ["AWS_REQUEST_PAYER"] = "requester" 

client = Client()
client

In [ ]:
@dask.delayed
def tif_to_da(tif):
    return xr.open_rasterio('s3://'+tif, chunks={'band':1, 'x':512, 'y':512})

In [ ]:
%%time
tifs = ets
lazy_da =[tif_to_da(tif) for tif in tifs]
dalist = dask.compute(*lazy_da)
da = xr.concat(dalist, dim='band')
da = da.rename({'band':'yearday'})

In [ ]:
!aws s3 ls --request-payer requester dev-et-data/tiles/tile40N-110E_chip40N-110E_o/etasw_2014059.tif

In [ ]:
da

In [ ]:
da = da.assign_coords(yearday=range(2014001,2014366))

In [ ]:
da

In [ ]:
ds = da.to_dataset(name='etasw')

In [ ]:
#ds.sel(yearday=2014001).etasw.load().hvplot(grid=True, width=962, height=962, cmap='viridis')
ds.sel(yearday=2014001).etasw.hvplot(grid=True, width=962, height=962, cmap='viridis')

In [ ]:
ds.sel(yearday=2014180).etasw.hvplot(grid=True, width=962, height=962, cmap='viridis')

In [ ]:
chip='dev-et-data/tiles/tile40N-110E_chip40N-110E'
ets=[]

for i in range(1,365+1):
    day = '{:03}'.format(i)
    #print(day)
    tif = chip + '/' + 'etasw_2014' + day + '.tif'
    #print(tif)
    ets.append(tif)

In [ ]:
ets

In [ ]:
%%time
tifs = ets
lazy_da2 =[tif_to_da(tif) for tif in tifs]
dalist2 = dask.compute(*lazy_da2)
da2 = xr.concat(dalist, dim='band')
da2 = da2.rename({'band':'yearday'})

In [ ]:
da2 = da2.assign_coords(yearday=range(2014001,2014366))

In [ ]:
da2

In [ ]:
ds2 = da2.to_dataset(name='etasw')

In [ ]:
ds2.sel(yearday=2014001).etasw.hvplot(grid=True, width=962, height=962, cmap='viridis')

In [ ]:
ds2.sel(yearday=2014180).etasw.hvplot(grid=True, width=962, height=962, cmap='viridis')

In [ ]:
ary2 = ds2.sel(yearday=2014180).etasw.values

In [ ]:
ary2.shape

In [ ]:
ary = ds.sel(yearday=2014180).etasw.values
# ary = ds2.sel(yearday=2014180).etasw.values - sanity check only - 

In [ ]:
ary.shape

In [ ]:
import numpy as np
np.array_equal(ary,ary2)

In [ ]:
ary[np.isnan(ary)] = 0

In [ ]:
ary.shape

In [ ]:
ary2[np.isnan(ary2)] = 0

In [ ]:
np.array_equal(ary,ary2)